In [1]:
import numpy as np
import pandas as pd
import os, sys

import json

import urllib.parse
from urllib.parse import unquote
from urllib.parse import urlparse

# Version V

##  XGBoost only

### Загружаем данные

In [2]:
#Параметры вычислений
file_path = '~/project/fetch.csv'
file_limit = None

In [3]:
# Загружаем файл
theFetch = pd.read_csv(file_path, sep=',', nrows=file_limit  )

In [4]:
#формируем мега признак
theFetch['target'] = theFetch.gender+theFetch.age
theFetch.drop(['gender', 'age', 'неизвестный прочее'], axis=1, inplace=True )
theFetch.set_index(['uid'], inplace=True)

In [5]:
#Создаем карту групп признаков: где чего лежит
theTargetName = sorted(theFetch.target.unique())
theTargetMap = pd.DataFrame( {'code':range(0, len(theTargetName)) }, index = theTargetName )

In [6]:
#Генерируем номера групп согластно карте признаков
theFetch['targetID'] = theFetch['target'].apply( lambda x:  theTargetMap.code.loc[x] )
theFetch.drop(['target'], axis=1, inplace=True )
theFetch.sort_values(by=['targetID'], inplace=True)
theFetch.fillna(0, inplace=True)
theFetch.columns = ['P' + str(i) for i in range(1, len(theFetch.columns))] + ['targetID']

In [7]:
#Рассчитываем положения их смещения в общем массиве
theTargetMap['len'] = theTargetMap['code'].apply( lambda type:  len(theFetch[theFetch.targetID == type]) )
theTargetMap['begin'] = [theTargetMap[theTargetMap.code < type ]['len'].sum() \
                        if type > 0 else 0 \
                        for type in range(0, 11) ]
theTargetMap['end'] = [theTargetMap[theTargetMap.code <= type ]['len'].sum() \
                        if type > 0 else int(theTargetMap[theTargetMap.code == type ]['len']) \
                        for type in range(0, 11) ]

In [8]:
theFetch.reset_index( inplace=True)
theFetch.set_index(['uid', 'targetID'], inplace=True)

for column in theFetch.columns:
    mean = theFetch[column].mean();
    std = theFetch[column].std();
    theFetch[column] = (theFetch[column]-mean)/std

In [9]:
#Эта тупая тварь не сохраняет в файл индексы. Ставим заплатку
theFetch.reset_index(inplace=True)
theFetch.to_csv('~/project/xgb_fetch.csv', sep=',', index=False)
theFetch.to_csv('~/project/xgb_fetch_map.csv', sep=',', index=False)
theFetch.set_index(['uid', 'targetID'], inplace=True)

### Любуемся результатом

In [10]:
theTargetMap

,code,len,begin,end
--,0,5000,0,5000
F18-24,1,2886,5000,7886
F25-34,2,6791,7886,14677
F35-44,3,4271,14677,18948
F45-54,4,2597,18948,21545
F>=55,5,895,21545,22440
M18-24,6,2012,22440,24452
M25-34,7,8666,24452,33118
M35-44,8,5089,33118,38207
M45-54,9,2147,38207,40354


In [11]:
theFetch.head()

,,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,...,P75,P76,P77,P78,P79,P80,P81,P82,P83,P84
uid,targetID,,,,,,,,,,,,,,,,,,,,,
fe1e01f3-5877-4a34-a300-3cfffc2f48e1,0,-0.038549,-0.013931,-0.022624,-0.022735,-0.025695,-0.005708,-0.06966,-0.010294,-0.02052,-0.061206,...,-0.069087,-0.00854,-0.048664,-0.05083,-0.044051,-0.008257,-0.047558,-0.026198,-0.012463,-0.008051
50e7982d-9e06-4749-9e53-058a1bfec672,0,-0.038549,-0.013931,-0.022624,-0.022735,-0.025695,-0.005708,-0.06966,-0.010294,-0.02052,-0.061206,...,-0.069087,-0.00854,-0.048664,-0.05083,-0.044051,-0.008257,-0.047558,-0.026198,-0.012463,-0.008051
50e9c5b7-f657-4862-88b7-11e4daf3e9e8,0,-0.038549,-0.013931,-0.022624,-0.022735,-0.025695,-0.005708,-0.06966,-0.010294,-0.02052,-0.061206,...,-0.069087,-0.00854,-0.048664,-0.05083,-0.044051,-0.008257,-0.047558,-0.026198,-0.012463,-0.008051
50ec4943-188c-42ae-ba1e-28b6aa488eb2,0,-0.038549,-0.013931,-0.022624,-0.022735,-0.025695,-0.005708,-0.06966,-0.010294,-0.02052,-0.061206,...,-0.069087,-0.00854,-0.048664,-0.05083,-0.044051,-0.008257,-0.047558,-0.026198,-0.012463,-0.008051
50ec49bd-a239-4a34-9001-159ffc7b98fe,0,-0.038549,-0.013931,-0.022624,-0.022735,-0.025695,-0.005708,-0.06966,-0.010294,-0.02052,-0.061206,...,-0.069087,-0.00854,-0.048664,-0.05083,-0.044051,-0.008257,0.364307,-0.026198,-0.012463,-0.008051


In [12]:
theFetch.describe()

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,...,P75,P76,P77,P78,P79,P80,P81,P82,P83,P84
count,4.113800e+04,4.113800e+04,4.113800e+04,4.113800e+04,4.113800e+04,4.113800e+04,4.113800e+04,4.113800e+04,4.113800e+04,4.113800e+04,...,4.113800e+04,4.113800e+04,4.113800e+04,4.113800e+04,4.113800e+04,4.113800e+04,4.113800e+04,4.113800e+04,4.113800e+04,4.113800e+04
mean,8.158214e-15,4.088951e-17,-4.123065e-15,-3.864502e-15,-8.411444e-15,6.979859e-16,2.396811e-15,-7.712418e-16,5.278886e-15,-8.761442e-16,...,1.617893e-15,2.203340e-15,-8.951069e-17,2.848808e-15,-6.524835e-15,3.755825e-15,-1.301012e-14,-2.018962e-15,-7.651581e-16,-4.950041e-15
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-3.854878e-02,-1.393146e-02,-2.262425e-02,-2.273501e-02,-2.569470e-02,-5.708429e-03,-6.966028e-02,-1.029442e-02,-2.051984e-02,-6.120557e-02,...,-6.908749e-02,-8.539837e-03,-4.866369e-02,-5.082998e-02,-4.405108e-02,-8.256803e-03,-4.755806e-02,-2.619758e-02,-1.246266e-02,-8.051403e-03
25%,-3.854878e-02,-1.393146e-02,-2.262425e-02,-2.273501e-02,-2.569470e-02,-5.708429e-03,-6.966028e-02,-1.029442e-02,-2.051984e-02,-6.120557e-02,...,-6.908749e-02,-8.539837e-03,-4.866369e-02,-5.082998e-02,-4.405108e-02,-8.256803e-03,-4.755806e-02,-2.619758e-02,-1.246266e-02,-8.051403e-03
50%,-3.854878e-02,-1.393146e-02,-2.262425e-02,-2.273501e-02,-2.569470e-02,-5.708429e-03,-6.966028e-02,-1.029442e-02,-2.051984e-02,-6.120557e-02,...,-6.908749e-02,-8.539837e-03,-4.866369e-02,-5.082998e-02,-4.405108e-02,-8.256803e-03,-4.755806e-02,-2.619758e-02,-1.246266e-02,-8.051403e-03
75%,-3.854878e-02,-1.393146e-02,-2.262425e-02,-2.273501e-02,-2.569470e-02,-5.708429e-03,-6.966028e-02,-1.029442e-02,-2.051984e-02,-6.120557e-02,...,-6.908749e-02,-8.539837e-03,-4.866369e-02,-5.082998e-02,-4.405108e-02,-8.256803e-03,-4.755806e-02,-2.619758e-02,-1.246266e-02,-8.051403e-03
max,7.215634e+01,1.841297e+02,1.442205e+02,1.306503e+02,1.086789e+02,2.019283e+02,6.222783e+01,1.558118e+02,1.115405e+02,6.676245e+01,...,7.712934e+01,1.561301e+02,1.704394e+02,7.138234e+01,1.229949e+02,1.576949e+02,1.175812e+02,1.962614e+02,1.767769e+02,1.656013e+02


### Выделяем диапазон для исследований

In [13]:
predictData = (theFetch.values)[0:5000]
fullData = (theFetch.values)[5000:41138]
fullTarget = np.array( theFetch.index.get_level_values('targetID') )[5000:41138]
fullData.shape, fullTarget.shape 

((36138, 84), (36138,))

### Загоняем в xgboost

In [14]:
%%time
from sklearn.model_selection import train_test_split
#Делим мастер данные на массивы
masterData, researchmentData, masterTarget, researchmentTarget = \
    train_test_split(fullData, fullTarget, test_size= 0.33, random_state=33 )

CPU times: user 348 ms, sys: 80 ms, total: 428 ms
Wall time: 431 ms


In [15]:
%%time
import xgboost

# У кого xgboost длиннее тот и прав
model = xgboost.XGBRegressor(n_estimators = 200, nthread= -1, max_depth = 6, objective = 'multi:softmax', num_class = 11 ) #
model.fit(masterData, masterTarget)
print( model.score(masterData, masterTarget) )

-0.1946309931896415
CPU times: user 5min 29s, sys: 2min 46s, total: 8min 16s
Wall time: 16.1 s


In [16]:

researchmentLabel = model.predict(researchmentData)
researchmentPredict = model.predict(predictData)

In [19]:
from sklearn.metrics import classification_report
report = classification_report(researchmentTarget, researchmentLabel, target_names=list(theTargetMap.index[1:])) #['...', '...']
print(report)

              precision    recall  f1-score   support

      F18-24       0.26      0.05      0.08       986
      F25-34       0.26      0.33      0.29      2195
      F35-44       0.25      0.09      0.13      1408
      F45-54       0.15      0.02      0.03       797
       F>=55       0.17      0.00      0.01       305
      M18-24       0.20      0.01      0.01       663
      M25-34       0.28      0.73      0.41      2897
      M35-44       0.29      0.13      0.18      1699
      M45-54       0.15      0.02      0.04       707
       M>=55       0.00      0.00      0.00       269

    accuracy                           0.27     11926
   macro avg       0.20      0.14      0.12     11926
weighted avg       0.24      0.27      0.21     11926



In [20]:
researchmentPredict

array([7., 6., 7., ..., 7., 7., 7.], dtype=float32)